In [1]:
!pip3 install pymysql==0.9.3

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [109]:
import boto3
import pandas as pd
import numpy as np
import s3fs

In [110]:
path1='s3://productscsv/hp_laptops.csv'
df=pd.read_csv(path1)
df

,Image,alinknormal_URL,Title,adeclarative,asizebase,aoffscreen,Price,acolorbase,arow2,arow,...,Price1,alinknormal_URL.1,alinknormal,Like.1,Like5,Like6,Like7,Like8,Like9,Like10
0,NaN,NaN,NaN,4.4 out of 5 stars.,NaN,Get Fast Performance & Smooth Video Calls,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://m.media-amazon.com/images/I/71B71IA8sG...,https://www.amazon.com/gp/slredirect/picassoRe...,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,4.4 out of 5 stars,149,$849.00,849\n.,"Tue, May 3",Get it as soon as,FREE Shipping by Amazon,...,NaN,NaN,NaN,Display Size,17.3 inches,Disk Size,1 TB,RAM,32 GB,Operating System
2,https://m.media-amazon.com/images/I/51uG6H2MLz...,https://www.amazon.com/gp/slredirect/picassoRe...,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",4.4 out of 5 stars,14,$899.00,899\n.,"Tue, May 3",NaN,NaN,...,NaN,NaN,NaN,Display Size,15.6 inches,Disk Size,256 GB,RAM,16 GB,Operating System
3,https://m.media-amazon.com/images/I/61Z-ANWd3b...,https://www.amazon.com/HP-Premium-14-inch-Cele...,2022 Newest HP Premium 14-inch HD Laptop| Inte...,4.5 out of 5 stars,42,$359.99,359\n.,"Tue, May 3",Get it as soon as,FREE Shipping by Amazon,...,NaN,NaN,NaN,Display Size,14 inches,Disk Size,64 GB,RAM,8.0 GB,Operating System
4,https://m.media-amazon.com/images/I/718V0aqaE1...,https://www.amazon.com/HP-Pavilion-Computer-i3...,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",4.5 out of 5 stars,79,$519.00,519\n.,"Mon, May 2",Get it as soon as,FREE Shipping by Amazon,...,NaN,NaN,NaN,Display Size,17.3 inches,Disk Size,256 GB,RAM,12 GB,Operating System
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4416,https://m.media-amazon.com/images/I/81EUotWsQw...,https://www.amazon.com/HP-255-15-6-LCD-Noteboo...,"HP 255 G7 15.6"" LCD Notebook - AMD (7th Gen) A...",4.5 out of 5 stars,4,$528.82,528\n.,"Tue, May 3",NaN,FREE Shipping,...,NaN,NaN,NaN,Display Size,15.6 inches,Disk Size,0 GB,RAM,8 GB,Operating System
4417,https://m.media-amazon.com/images/I/71fCSwvM82...,https://www.amazon.com/HP-EliteBook-i5-4300U-C...,"HP EliteBook 840 G1 Business Laptop, 14in HD+ ...",4.1 out of 5 stars,11,$409.80,409\n.,"Fri, May 6",NaN,FREE Shipping,...,$409.78,https://www.amazon.com/gp/offer-listing/B07GCR...,(3 new offers),Display Size,14 inches,Disk Size,240 GB,RAM,8 GB,Operating System
4418,https://m.media-amazon.com/images/I/31g5SHwNc1...,https://www.amazon.com/HP-Folio-Laptop-Process...,HP Folio 9470M Laptop (Intel Core i5 Processor...,4.0 out of 5 stars,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Display Size,14 inches,Disk Size,320 GB,RAM,8 GB,Operating System
4419,https://m.media-amazon.com/images/I/81pDlK3Qhb...,https://www.amazon.com/gp/slredirect/picassoRe...,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",4.6 out of 5 stars,16,"$1,029.00","1,029\n.","Wed, May 4",NaN,NaN,...,NaN,NaN,NaN,Display Size,15.6 inches,Disk Size,-,RAM,16.0 GB,Operating System


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4421 entries, 0 to 4420
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Image              4122 non-null   object
 1   alinknormal_URL    4122 non-null   object
 2   Title              4122 non-null   object
 3   adeclarative       4266 non-null   object
 4   asizebase          4121 non-null   object
 5   aoffscreen         3882 non-null   object
 6   Price              3747 non-null   object
 7   acolorbase         3714 non-null   object
 8   arow2              1097 non-null   object
 9   arow               2630 non-null   object
 10  Like               1165 non-null   object
 11  Price1             832 non-null    object
 12  alinknormal_URL.1  832 non-null    object
 13  alinknormal        832 non-null    object
 14  Like.1             3969 non-null   object
 15  Like5              3969 non-null   object
 16  Like6              3969 non-null   object


In [112]:
new_df=df[['Title','Image','adeclarative','alinknormal_URL','asizebase','aoffscreen','Price1','Like']] 

In [113]:
#Removing sponsored ad wordings from df
df1=new_df[~new_df.adeclarative.str.contains("Sponsored\n",na=False)]
df1

,Title,Image,adeclarative,alinknormal_URL,asizebase,aoffscreen,Price1,Like
0,NaN,NaN,4.4 out of 5 stars.,NaN,NaN,Get Fast Performance & Smooth Video Calls,NaN,NaN
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149,$849.00,NaN,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14,$899.00,NaN,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42,$359.99,NaN,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79,$519.00,NaN,NaN
...,...,...,...,...,...,...,...,...
4416,"HP 255 G7 15.6"" LCD Notebook - AMD (7th Gen) A...",https://m.media-amazon.com/images/I/81EUotWsQw...,4.5 out of 5 stars,https://www.amazon.com/HP-255-15-6-LCD-Noteboo...,4,$528.82,NaN,NaN
4417,"HP EliteBook 840 G1 Business Laptop, 14in HD+ ...",https://m.media-amazon.com/images/I/71fCSwvM82...,4.1 out of 5 stars,https://www.amazon.com/HP-EliteBook-i5-4300U-C...,11,$409.80,$409.78,More Buying Choices
4418,HP Folio 9470M Laptop (Intel Core i5 Processor...,https://m.media-amazon.com/images/I/31g5SHwNc1...,4.0 out of 5 stars,https://www.amazon.com/HP-Folio-Laptop-Process...,1,NaN,NaN,NaN
4419,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",https://m.media-amazon.com/images/I/81pDlK3Qhb...,4.6 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,16,"$1,029.00",NaN,NaN


In [114]:
#Renaming columns withn readble columns names
df1=df1.rename(
        columns={
            'adeclarative':'Rating',
            'Image':'Image',
            'alinknormal_URL':'ProdLink',
            'asizebase':'TotalReviews',
            'aoffscreen':'Price',
            'Price1':'OrgPrice',
            'Like':'Coupons'
               
        })

In [115]:
df1.head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
0,NaN,NaN,4.4 out of 5 stars.,NaN,NaN,Get Fast Performance & Smooth Video Calls,NaN,NaN
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149,$849.00,NaN,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14,$899.00,NaN,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42,$359.99,NaN,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79,$519.00,NaN,NaN


In [116]:
#Remove white space
df = df1.applymap(lambda x: x.strip() if isinstance(x, str) else x) 

In [117]:
cols = df1.select_dtypes(['object']).columns
df1[cols] = df1[cols].apply(lambda x: x.str.strip())
#print(df1.head())
df1.tail()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
4416,"HP 255 G7 15.6"" LCD Notebook - AMD (7th Gen) A...",https://m.media-amazon.com/images/I/81EUotWsQw...,4.5 out of 5 stars,https://www.amazon.com/HP-255-15-6-LCD-Noteboo...,4,$528.82,NaN,NaN
4417,"HP EliteBook 840 G1 Business Laptop, 14in HD+ ...",https://m.media-amazon.com/images/I/71fCSwvM82...,4.1 out of 5 stars,https://www.amazon.com/HP-EliteBook-i5-4300U-C...,11,$409.80,$409.78,More Buying Choices
4418,HP Folio 9470M Laptop (Intel Core i5 Processor...,https://m.media-amazon.com/images/I/31g5SHwNc1...,4.0 out of 5 stars,https://www.amazon.com/HP-Folio-Laptop-Process...,1,NaN,NaN,NaN
4419,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",https://m.media-amazon.com/images/I/81pDlK3Qhb...,4.6 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,16,"$1,029.00",NaN,NaN
4420,"2022 HP Pavilion 17 Laptop, 17.3"" HD+ Anti-Gla...",https://m.media-amazon.com/images/I/81y-IYthAy...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,28,$599.99,NaN,NaN


In [118]:
#taking only not null from image, prodlink,title..basically removing null values from title 
df = df1[df1['Image'].notna()]
df = df1[df1['ProdLink'].notna()]
df = df1[df1['Title'].notna()]
df.head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149,$849.00,NaN,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14,$899.00,NaN,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42,$359.99,NaN,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79,$519.00,NaN,NaN
5,"HP 15-inch Laptop, 11th Generation Intel Core ...",https://m.media-amazon.com/images/I/71RD3vsjIY...,4.4 out of 5 stars,https://www.amazon.com/HP-Generation-i5-1135G7...,"3,339",$520.00,$460.00,-21%


In [119]:
df['TotalReviews'] = df['TotalReviews'].replace(np.nan, 0)

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/1149219182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalReviews'] = df['TotalReviews'].replace(np.nan, 0)


#Fill Missing Values of price and original price vice versa
df.Price.fillna(df.OrgPrice, inplace=True)
df.OrgPrice.fillna(df.Price, inplace=True)
df.head()

In [120]:
#Fill Missing Values of price and original price vice versa
df.Price.fillna(df.OrgPrice, inplace=True)
df.OrgPrice.fillna(df.Price, inplace=True)
df.head(10)

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/520675218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Price.fillna(df.OrgPrice, inplace=True)
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/520675218.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.OrgPrice.fillna(df.Price, inplace=True)


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149,$849.00,$849.00,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14,$899.00,$899.00,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42,$359.99,$359.99,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79,$519.00,$519.00,NaN
5,"HP 15-inch Laptop, 11th Generation Intel Core ...",https://m.media-amazon.com/images/I/71RD3vsjIY...,4.4 out of 5 stars,https://www.amazon.com/HP-Generation-i5-1135G7...,"3,339",$520.00,$460.00,-21%
6,"HP Pavilion 15 Laptop, 11th Gen Intel Core i7-...",https://m.media-amazon.com/images/I/711Nx6ZoRM...,4.2 out of 5 stars,https://www.amazon.com/HP-i7-1165G7-Processor-...,255,$833.36,$666.34,More Buying Choices
7,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",https://m.media-amazon.com/images/I/81pDlK3Qhb...,4.6 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,16,"$1,529.00","$1,529.00",NaN
8,HP Stream 9VK97UA#ABA 14 inches HD(1366x768) D...,https://m.media-amazon.com/images/I/71sOrI1hB9...,4.4 out of 5 stars,https://www.amazon.com/HP-1366x768-Dual-Core-P...,838,$185.00,$185.00,-14%
9,"Newest 2022 HP 15.6"" FHD Micro-Edge Laptop, AM...",https://m.media-amazon.com/images/I/71CeaGJHH+...,4.3 out of 5 stars,https://www.amazon.com/HP-Micro-Edge-i7-1160G7...,119,$659.00,$659.00,NaN
10,"2022 HP Pavilion Laptop, 14-inch HD Touchscree...",https://m.media-amazon.com/images/I/81RBQ5VUem...,4.4 out of 5 stars,https://www.amazon.com/HP-Pavilion-Touchscreen...,38,$399.99,$399.99,NaN


In [121]:
#Removing Commas in the  Total Reviews and converting them to float
df['TotalReviews'] = df['TotalReviews'].str.replace(',', '').replace(np.nan,0)
df.tail()

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/1927146474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalReviews'] = df['TotalReviews'].str.replace(',', '').replace(np.nan,0)


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
4416,"HP 255 G7 15.6"" LCD Notebook - AMD (7th Gen) A...",https://m.media-amazon.com/images/I/81EUotWsQw...,4.5 out of 5 stars,https://www.amazon.com/HP-255-15-6-LCD-Noteboo...,4,$528.82,$528.82,NaN
4417,"HP EliteBook 840 G1 Business Laptop, 14in HD+ ...",https://m.media-amazon.com/images/I/71fCSwvM82...,4.1 out of 5 stars,https://www.amazon.com/HP-EliteBook-i5-4300U-C...,11,$409.80,$409.78,More Buying Choices
4418,HP Folio 9470M Laptop (Intel Core i5 Processor...,https://m.media-amazon.com/images/I/31g5SHwNc1...,4.0 out of 5 stars,https://www.amazon.com/HP-Folio-Laptop-Process...,1,NaN,NaN,NaN
4419,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",https://m.media-amazon.com/images/I/81pDlK3Qhb...,4.6 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,16,"$1,029.00","$1,029.00",NaN
4420,"2022 HP Pavilion 17 Laptop, 17.3"" HD+ Anti-Gla...",https://m.media-amazon.com/images/I/81y-IYthAy...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,28,$599.99,$599.99,NaN


In [122]:
#Print Missing Values for reference
#coupon column, NAN
df[df.isnull().any(axis=1)].head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149,$849.00,$849.00,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14,$899.00,$899.00,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42,$359.99,$359.99,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79,$519.00,$519.00,NaN
7,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",https://m.media-amazon.com/images/I/81pDlK3Qhb...,4.6 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,16,"$1,529.00","$1,529.00",NaN


In [123]:
#Removing Commas in the  Total Reviews and converting them to float
df['TotalReviews'] = df['TotalReviews'].str.replace(',', '').astype(float)
df.tail()

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/4275027062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalReviews'] = df['TotalReviews'].str.replace(',', '').astype(float)


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
4416,"HP 255 G7 15.6"" LCD Notebook - AMD (7th Gen) A...",https://m.media-amazon.com/images/I/81EUotWsQw...,4.5 out of 5 stars,https://www.amazon.com/HP-255-15-6-LCD-Noteboo...,4.0,$528.82,$528.82,NaN
4417,"HP EliteBook 840 G1 Business Laptop, 14in HD+ ...",https://m.media-amazon.com/images/I/71fCSwvM82...,4.1 out of 5 stars,https://www.amazon.com/HP-EliteBook-i5-4300U-C...,11.0,$409.80,$409.78,More Buying Choices
4418,HP Folio 9470M Laptop (Intel Core i5 Processor...,https://m.media-amazon.com/images/I/31g5SHwNc1...,4.0 out of 5 stars,https://www.amazon.com/HP-Folio-Laptop-Process...,1.0,NaN,NaN,NaN
4419,"HP Pavilion 15 Windows 10 Pro Business Laptop,...",https://m.media-amazon.com/images/I/81pDlK3Qhb...,4.6 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,16.0,"$1,029.00","$1,029.00",NaN
4420,"2022 HP Pavilion 17 Laptop, 17.3"" HD+ Anti-Gla...",https://m.media-amazon.com/images/I/81y-IYthAy...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,28.0,$599.99,$599.99,NaN


In [88]:
df.dtypes

Title            object
Image            object
Rating           object
ProdLink         object
TotalReviews    float64
Price            object
OrgPrice         object
Coupons          object
dtype: object

In [125]:
df['Price'] = df['Price'].str.replace('$','')
df['OrgPrice']=df['OrgPrice'].str.replace('$','')
df.head()

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/2555454753.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('$','')
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/2555454753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = df['Price'].str.replace('$','')
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/2555454753.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149.0,849.00,849.00,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14.0,899.00,899.00,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42.0,359.99,359.99,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79.0,519.00,519.00,NaN
5,"HP 15-inch Laptop, 11th Generation Intel Core ...",https://m.media-amazon.com/images/I/71RD3vsjIY...,4.4 out of 5 stars,https://www.amazon.com/HP-Generation-i5-1135G7...,3339.0,520.00,460.00,-21%


In [128]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['OrgPrice'] = pd.to_numeric(df['OrgPrice'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4122 entries, 1 to 4420
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         4122 non-null   object 
 1   Image         4122 non-null   object 
 2   Rating        4121 non-null   object 
 3   ProdLink      4122 non-null   object 
 4   TotalReviews  4121 non-null   float64
 5   Price         3112 non-null   float64
 6   OrgPrice      3164 non-null   float64
 7   Coupons       1165 non-null   object 
dtypes: float64(3), object(5)
memory usage: 289.8+ KB


/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/926543028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_34592/926543028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OrgPrice'] = pd.to_numeric(df['OrgPrice'], errors='coerce')


In [131]:
df.head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149.0,849.00,849.00,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14.0,899.00,899.00,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42.0,359.99,359.99,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79.0,519.00,519.00,NaN
5,"HP 15-inch Laptop, 11th Generation Intel Core ...",https://m.media-amazon.com/images/I/71RD3vsjIY...,4.4 out of 5 stars,https://www.amazon.com/HP-Generation-i5-1135G7...,3339.0,520.00,460.00,-21%


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,HP Pavilion 17.3-inch IPS FHD Laptop (2022 Mod...,https://m.media-amazon.com/images/I/71B71IA8sG...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,149.0,849.00,849.00,NaN
2,"2022 Newest HP 15 Laptop, 15.6-inch Full HD Di...",https://m.media-amazon.com/images/I/51uG6H2MLz...,4.4 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,14.0,899.00,899.00,NaN
3,2022 Newest HP Premium 14-inch HD Laptop| Inte...,https://m.media-amazon.com/images/I/61Z-ANWd3b...,4.5 out of 5 stars,https://www.amazon.com/HP-Premium-14-inch-Cele...,42.0,359.99,359.99,NaN
4,"2022 Newest HP Pavilion 17.3"" FHD Display Lapt...",https://m.media-amazon.com/images/I/718V0aqaE1...,4.5 out of 5 stars,https://www.amazon.com/HP-Pavilion-Computer-i3...,79.0,519.00,519.00,NaN
5,"HP 15-inch Laptop, 11th Generation Intel Core ...",https://m.media-amazon.com/images/I/71RD3vsjIY...,4.4 out of 5 stars,https://www.amazon.com/HP-Generation-i5-1135G7...,3339.0,520.00,460.00,-21%


ValueError: could not convert string to float: 'Get Fast Performance & Smooth Video Calls'

In [104]:
df['Price'] = df['Price'].where(df['Price'] != '', 0)
df['OrgPrice'] = df['OrgPrice'].where(df['OrgPrice'] != '', 0)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4421 entries, 0 to 4420
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         4122 non-null   object 
 1   Image         4122 non-null   object 
 2   Rating        4266 non-null   object 
 3   ProdLink      4122 non-null   object 
 4   TotalReviews  4121 non-null   float64
 5   Price         3882 non-null   object 
 6   OrgPrice      3882 non-null   object 
 7   Coupons       1165 non-null   object 
dtypes: float64(1), object(7)
memory usage: 310.9+ KB


ValueError: could not convert string to float: 'Get Fast Performance & Smooth Video Calls'

In [103]:
df['Price'] = pd.to_numeric(df['Price'])
df['OrgPrice'] = pd.to_numeric(df['OrgPrice'])

ValueError: Unable to parse string "Get Fast Performance & Smooth Video Calls" at position 0